## Trabajador experto en conocimiento

### Un agente que responde preguntas y es un trabajador experto en conocimiento
### Para ser utilizado por empleados de Insurellm, una empresa de tecnología de seguros
### El agente debe ser preciso y la solución debe ser de bajo costo.

Este proyecto utilizará RAG (Retrieval Augmented Generation, generación aumentada de recuperación) para garantizar que nuestro asistente de preguntas y respuestas tenga una alta precisión.

Esta primera implementación utilizará un tipo de RAG simple, de fuerza bruta.

In [15]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# imports de langchain, plotly y Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

Matplotlib is building the font cache; this may take a moment.


In [3]:
# El precio es un factor para nuestra empresa, por eso vamos a utilizar un modelo de bajo costo.

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
# Cargar variables de entorno en un archivo llamado .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
# Leer documentos usando los cargadores de LangChain
# Tomar todo lo que está en todas las subcarpetas de nuestra base de conocimiento

folders = glob.glob("knowledge-base/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

# Con agradecimientos a CG y Jon R, estudiantes del curso, por esta solución necesaria para algunos usuarios.
text_loader_kwargs = {'encoding': 'utf-8'}
# Si eso no funciona, algunos usuarios de Windows podrían necesitar descomentar la siguiente línea.
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Created a chunk of size 1252, which is longer than the specified 1000


Total number of chunks: 137
Document types found: {'company', 'contracts', 'products', 'employees'}


## Una nota al margen sobre las incrustaciones y los "LLM de codificación automática"

Asignaremos cada fragmento de texto a un vector que representa el significado del texto, conocido como incrustación.

OpenAI ofrece un modelo para hacer esto, que utilizaremos llamando a su API con un código LangChain.

Este modelo es un ejemplo de un "LLM de codificación automática" que genera una salida dada una entrada completa.
Es diferente a todos los demás LLM que hemos analizado hoy, que se conocen como "LLM autorregresivos", y generan tokens futuros basados ​​solo en el contexto pasado.

Otro ejemplo de un LLM de codificación automática es BERT de Google. Además de la incrustación, los LLM de codificación automática se utilizan a menudo para la clasificación.

### Nota al margen

En la semana 8 volveremos a RAG y a las incrustaciones vectoriales, y utilizaremos un codificador vectorial de código abierto para que los datos nunca abandonen nuestra computadora; esa es una consideración importante cuando se crean sistemas empresariales y los datos deben permanecer internos.

In [6]:
# Coloque los fragmentos de datos en un almacén de vectores que asocie una incrustación de vectores con cada fragmento
# Chroma es una popular base de datos de vectores de código abierto basada en SQLLite

embeddings = OpenAIEmbeddings()

# Eliminar si ya existe

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Crear almacén de vectores

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 137 documents


In [8]:
# Investiguemos los vectores

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"Hay {count:,} vectores con {dimensions:,} dimensiones en el almacén de vectores")

Hay 137 vectores con 1,536 dimensiones en el almacén de vectores


## Visualización del almacén de vectores

Tomémonos un minuto para observar los documentos y sus vectores de incrustación para ver qué está sucediendo.

In [9]:
# Trabajo previo (¡con agradecimiento a Jon R por identificar y corregir un error en esto!)

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [ ]:
# ¡A los humanos nos resulta más fácil visualizar cosas en 2D!
# Reducir la dimensionalidad de los vectores a 2D usando t-SNE
# (incrustación de vecinos estocásticos distribuidos en t)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Visualización 2D Chroma Vector Store',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Visualización 3D Chroma Vector Store',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Es hora de usar LangChain para unirlo todo

In [10]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# juntando todo: configure la cadena de conversación con GPT 3.5 LLM, el almacén vectorial y la memoria
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [11]:
# Vamos a intentar una pregunta sencilla

query = "Por favor, explique qué es Insurellm en un par de frases."
result = conversation_chain.invoke({"question": query})
print(result["answer"])

Insurellm es una innovadora empresa de tecnología de seguros fundada en 2015 que ofrece soluciones de software para el sector de seguros, incluyendo productos como Carllm, Homellm, Rellm y Marketllm. Con 200 empleados y más de 300 clientes en todo el mundo, Insurellm busca revolucionar la industria de seguros a través de productos innovadores.


In [18]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## Ahora, lo mostraremos en Gradio usando la interfaz de Chat:

Una forma rápida y sencilla de crear un prototipo de chat con un LLM

In [17]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [19]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [20]:
# Investiguemos qué se envía detrás de escena

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

query = "¿Quién recibió el prestigioso premio IIOTY en 2023?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nRespuesta:", answer)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
## Historial de desempeño anual
- **2020:**
- Completó la incorporación con éxito.
- Cumplió con las expectativas en la entrega de los hitos del proyecto. - Recibí comentarios positivos de los líderes del equipo.

- **2021:**
- Logré una tasa de éxito del 95 % en los plazos de entrega de proyectos.
- Recibí el premio "Rising Star" en la gala anual de la empresa por contribuciones sobresalientes.

- **2022:**
- Superé los objetivos al optimizar el código backend existente, mejorando el rendimiento del sistema en un 25 %.
- Realicé sesiones de capacitación para desarrolladores junior, fomentando el intercambio de conocimientos.

- **2023:**
- Lideré una importante revisión de la arquitectura interna de la API, mejorando los protocolos de seguridad.
- Contribuí a la transición de la

In [27]:
# crear un nuevo Chat con OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Configurar la memoria de conversación para el chat.
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# el recuperador es una abstracción sobre el VectorStore que se utilizará durante RAG; k es la cantidad de fragmentos a utilizar
retriever = vectorstore.as_retriever(search_kwargs={"k": 60})

# juntando todo: configure la cadena de conversación con GPT 3.5 LLM, el almacén vectorial y la memoria
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [28]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [29]:
view = gr.ChatInterface(chat).launch()

/Users/juangabriel/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


# Ejercicios

Intenta aplicar esto a tu propia carpeta de datos, de modo que crees un trabajador del conocimiento personal, un experto en su propia información.
